# SVM기반 분류모델_당뇨병여부(지체장애인 유의미한 변수) 1차 시도

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE, ADASYN
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
df=pd.read_excel("pydata/[로지스틱] 유의한 변수 추출(지체장애 당뇨병).xlsx")
df.head()

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,16)만성질환명(골관절염(퇴행성_x000D_\n관절염)),17)만성질환명(류마티스 관절염),18)만성질환명(골다공증),19)만성질환명(척추측만증),20)만성질환명(허리목통증),21)만성질환명(피부염),22)만성질환명(백내장),23)만성질환명(우울증),24)만성질환명(암),25)만성질환명(기타)
0,1970,0,64,0,178.0,77.0,1,3,2,3,...,0,0,0,0,0,0,0,0,0,0
1,1949,30,69,0,163.0,55.0,1,2,1,3,...,0,0,0,0,0,0,0,0,0,1
2,1958,0,48,175,158.0,63.0,1,2,2,0,...,0,0,0,0,1,0,0,0,0,1
3,1955,30,48,175,163.0,62.0,1,5,2,2,...,0,0,0,0,1,0,0,1,0,0
4,1936,0,48,171,154.0,57.0,1,5,2,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
df.columns

Index(['생년', '월 혈압약 일수', '장애주된 원인', '질병명', '키', '몸무게', '현재 지속적 진료여부',
       '주관적 건강 평가', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부', '건강관련서비스 이용여부',
       '이용_만성질환관리', 'ADL_보행', '주로 도와주는 사람', '상지의지-소지', '자세보조용구-필요', '이동변기-소지',
       '이동변기-사용', '사용빈도', '1일 이용시간', '컴퓨터 사용 여부', '01)만성질환명(고혈압)',
       '02)만성질환명(뇌졸중,중풍)', '03)만성질환명(심근경색증)', '04)만성질환명(협심증)',
       '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)', '08)만성질환명(천식)',
       '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)', '11)만성질환명(위십이지장궤양)',
       '12)만성질환명(B형간염)', '13)만성질환명(C형간염)', '14)만성질환명(간경변증)', '15)만성질환명(신부전)',
       '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))', '17)만성질환명(류마티스 관절염)',
       '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)', '20)만성질환명(허리목통증)', '21)만성질환명(피부염)',
       '22)만성질환명(백내장)', '23)만성질환명(우울증)', '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [240]:
df1=df[['생년', '월 혈압약 일수', '장애주된 원인', '질병명', '키', '몸무게', '현재 지속적 진료여부',
       '주관적 건강 평가', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부', '건강관련서비스 이용여부',
       '이용_만성질환관리', 'ADL_보행', '주로 도와주는 사람', '상지의지-소지', '자세보조용구-필요', '이동변기-소지',
       '이동변기-사용', '사용빈도', '1일 이용시간', '컴퓨터 사용 여부', '06)만성질환명(당뇨병)']]
df1 #2724건 

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,64,0,178.0,77.0,1,3,2,3,...,1,0,2,2,2,0,0,0,1,0
1,1949,30,69,0,163.0,55.0,1,2,1,3,...,1,1,2,2,2,0,0,0,2,0
2,1958,0,48,175,158.0,63.0,1,2,2,0,...,1,0,2,2,2,0,0,0,2,0
3,1955,30,48,175,163.0,62.0,1,5,2,2,...,1,0,2,2,2,0,1,3,2,0
4,1936,0,48,171,154.0,57.0,1,5,2,0,...,4,13,2,2,2,0,2,9,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,1935,0,68,0,157.0,65.0,1,4,2,0,...,1,1,2,2,2,0,2,2,2,0
2720,1952,0,69,0,165.0,62.0,1,3,1,4,...,1,0,2,2,2,0,0,0,1,0
2721,1960,30,48,175,173.0,88.0,1,5,2,5,...,1,0,2,1,2,0,5,3,2,0
2722,1960,0,48,175,153.0,51.0,1,3,2,0,...,1,0,2,1,2,0,3,4,2,0


### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [241]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,48,171,154.0,57.0,1,5,2,0,...,4,13,2,2,2,0,2,9,2,1
7,1949,30,48,180,150.0,50.0,1,5,2,0,...,3,1,2,2,2,0,1,3,2,1
15,1967,0,40,103,161.0,70.0,1,4,2,0,...,2,3,2,2,2,0,3,4,2,1
26,1955,0,69,0,165.0,60.0,1,3,2,1,...,1,0,1,2,2,0,5,7,2,1
27,1944,0,69,0,168.0,74.0,1,4,2,1,...,1,3,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,62,0,163.0,57.0,1,3,2,0,...,1,1,2,2,2,0,0,0,1,1
2713,1938,30,48,171,150.7,67.0,1,4,2,0,...,1,0,2,2,2,0,1,4,2,1
2714,1932,30,69,0,152.0,59.0,1,5,2,0,...,1,0,2,2,2,0,1,3,2,1
2715,1938,0,48,180,145.0,60.0,1,4,2,1,...,1,3,2,2,2,0,0,0,2,1


In [277]:
out=df1.loc[[27]] # 당뇨명: 7, 27유 / 3, 24무  
out

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
27,1944,0,69,0,168.0,74.0,1,4,2,1,...,1,3,2,2,2,0,0,0,2,1


In [278]:
df2=df1.drop(out.index).reset_index(drop=True) # 7번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,64,0,178.0,77.0,1,3,2,3,...,1,0,2,2,2,0,0,0,1,0
1,1949,30,69,0,163.0,55.0,1,2,1,3,...,1,1,2,2,2,0,0,0,2,0
2,1958,0,48,175,158.0,63.0,1,2,2,0,...,1,0,2,2,2,0,0,0,2,0
3,1955,30,48,175,163.0,62.0,1,5,2,2,...,1,0,2,2,2,0,1,3,2,0
4,1936,0,48,171,154.0,57.0,1,5,2,0,...,4,13,2,2,2,0,2,9,2,1


In [279]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '장애주된 원인', '질병명', '키', '몸무게', '현재 지속적 진료여부',
       '주관적 건강 평가', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부', '건강관련서비스 이용여부',
       '이용_만성질환관리', 'ADL_보행', '주로 도와주는 사람', '상지의지-소지', '자세보조용구-필요', '이동변기-소지',
       '이동변기-사용', '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [280]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [281]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [282]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777, shuffle=y_data)

In [283]:
df.columns

Index(['생년', '월 혈압약 일수', '장애주된 원인', '질병명', '키', '몸무게', '현재 지속적 진료여부',
       '주관적 건강 평가', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부', '건강관련서비스 이용여부',
       '이용_만성질환관리', 'ADL_보행', '주로 도와주는 사람', '상지의지-소지', '자세보조용구-필요', '이동변기-소지',
       '이동변기-사용', '사용빈도', '1일 이용시간', '컴퓨터 사용 여부', '01)만성질환명(고혈압)',
       '02)만성질환명(뇌졸중,중풍)', '03)만성질환명(심근경색증)', '04)만성질환명(협심증)',
       '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)', '08)만성질환명(천식)',
       '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)', '11)만성질환명(위십이지장궤양)',
       '12)만성질환명(B형간염)', '13)만성질환명(C형간염)', '14)만성질환명(간경변증)', '15)만성질환명(신부전)',
       '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))', '17)만성질환명(류마티스 관절염)',
       '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)', '20)만성질환명(허리목통증)', '21)만성질환명(피부염)',
       '22)만성질환명(백내장)', '23)만성질환명(우울증)', '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

#### **오버 샘플링 적용**

In [284]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [285]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1422
1                1422
dtype: int64

In [286]:
print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

After OverSampling, the shape of train_X: (2844, 22)
After OverSampling, the shape of train_y: (2844, 1) 



In [287]:
categorical_features = ['월 혈압약 일수', '장애주된 원인', '질병명', '현재 지속적 진료여부',
       '주관적 건강 평가', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부', '건강관련서비스 이용여부',
       '이용_만성질환관리', 'ADL_보행', '주로 도와주는 사람', '상지의지-소지', '자세보조용구-필요', '이동변기-소지',
       '이동변기-사용', '사용빈도', '1일 이용시간', '컴퓨터 사용 여부'] # 범주형 변수

transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), categorical_features), remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

In [215]:
# 모델 생성
model = SVC(kernel='linear', C=1.0, probability=True)#,class_weight=class_weight)

#모델 학습
model.fit(x_sm, y_sm)

#모델 검증
print('training data fit :', model.score(x_sm, y_sm)) 
print('test data fit :', model.score(x_test, y_test)) 

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training data fit : 0.7454289732770746
test data fit : 0.5973072215422277


### SVC

In [177]:
# 그리드 서치
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), categorical_features), remainder='passthrough')
# # transformer = make_pipeline(transformer, MaxAbsScaler())

# pipe = Pipeline([('scaler', transformer),
#                  ('smote', SMOTE()),
#                 ('model', SVC())])

## 비대칭 매개변수 그리드 탐색
param_grid = [ {'kernel': ['rbf', 'ploy', 'sigmoid'], # 가장 성능이 좋은 것: 가우시안 RBF 커널
             'C': [0.01, 0.1, 1, 10], # 값이 클수록 이상치의 존재 가능성을 작게 봐서 좀 더 세심하게 결정 경계를 찾아낸다. -> 오버피팅(하드마진)
             'gamma': [0.01, 0.1, 1, 10]},  # 값이 클수록 결정 경계의 곡률이 커짐 -> 오버피팅
             {'kernel': ['linear'],
              'C': [0.01, 0.1, 1, 10]
             } ]

gs = GridSearchCV(estimator=SVC(probability=True), 
                    param_grid=param_grid, scoring='accuracy', cv = 5, n_jobs=-1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_))
print('최적 점수: {}'.format(gs.best_score_))
print(result.best_estimator_)

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 260.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 342, in _sparse_fit
    kernel_type = self._sparse_kernels.index(kernel)
Va

최적 파라미터: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
최적 점수: 0.7270237104184011
SVC(C=10, gamma=0.1, probability=True)


In [216]:
#모델 학습
svc_model = gs.best_estimator_

In [217]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1) # out 데이터에서 '당뇨병' 칼럼만 제거
x_test

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,이용_만성질환관리,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
27,1944,0,69,0,168.0,74.0,1,4,2,1,...,1,1,3,2,2,2,0,0,0,2


In [218]:
labels = ['무', '유']

In [219]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.8458560307869374


---
### RandomForest

In [232]:
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier()

#모델 학습
rfc_model.fit(x_sm, y_sm)

#모델 검증
print('training data fit :', rfc_model.score(x_sm, y_sm)) 
print('test data fit :', rfc_model.score(x_test, y_test)) 

<ipython-input-232-4f36d80b37a7>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc_model.fit(x_sm, y_sm)


training data fit : 1.0
test data fit : 0.6389228886168911


In [253]:
# 그리드 서치
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), categorical_features), remainder='passthrough')
# # transformer = make_pipeline(transformer, MaxAbsScaler())

# pipe = Pipeline([('scaler', transformer),
#                  ('smote', SMOTE()),
#                 ('model', SVC())])

param_grid = {'n_estimators': [10, 100, 200], # 결정트리 개수
             'max_features': ['sqrt', 'log'], # 각 노드에거 랜덤하게 선택할 '후보 특성' 개수 (분류: sqrt, 회귀: log)
             'max_depth': [3, 5, 7]
             }

gs = GridSearchCV(estimator = RandomForestClassifier(), cv= 5,
                 param_grid=param_grid, n_jobs = -1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_))
print('최적 점수: {}'.format(gs.best_score_))
print(result.best_estimator_)

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\

최적 파라미터: {'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 100}
최적 점수: 0.7025538998490062
RandomForestClassifier(max_depth=7, max_features='sqrt')


In [254]:
#모델 학습
rfc_model = gs.best_estimator_

In [255]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1) # out 데이터에서 '당뇨병' 칼럼만 제거
x_test

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,이용_만성질환관리,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
27,1944,0,69,0,168.0,74.0,1,4,2,1,...,1,1,3,2,2,2,0,0,0,2


In [256]:
labels = ['무', '유']

In [257]:
x_test = transformer.transform(x_test)

y_predict = rfc_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = rfc_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.5642879965882831


---
### LightXGB

In [271]:
import lightgbm as lgbm

lgb_model= lgbm.LGBMClassifier()
lgb_model.fit(x_sm, y_sm)

#모델 검증
print('training data fit :', lgb_model.score(x_sm, y_sm)) 
print('test data fit :', lgb_model.score(x_test, y_test)) 

training data fit : 0.9388185654008439
test data fit : 0.6413708690330477


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [272]:
# 그리드 서치
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), categorical_features), remainder='passthrough')
# # transformer = make_pipeline(transformer, MaxAbsScaler())

# pipe = Pipeline([('scaler', transformer),
#                  ('smote', SMOTE()),
#                 ('model', SVC())])

param_grid = {'learning_rate': [0.05, 0.1, 0.2],
             'max_depth': [3, 5, 7]}
gs = GridSearchCV(estimator=lgbm.LGBMClassifier(), param_grid=param_grid, cv=5, n_jobs=-1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'learning_rate': 0.2, 'max_depth': 7}
최적 점수: 0.7700623777816282
LGBMClassifier(learning_rate=0.2, max_depth=7)


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [273]:
#모델 학습
lgb_model = gs.best_estimator_

In [274]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1) # out 데이터에서 '당뇨병' 칼럼만 제거
x_test

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,이용_만성질환관리,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
27,1944,0,69,0,168.0,74.0,1,4,2,1,...,1,1,3,2,2,2,0,0,0,2


In [275]:
labels = ['무', '유']

In [276]:
x_test = transformer.transform(x_test)

y_predict = lgb_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = lgb_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.5404505814157112


---
### Logistic Regresstion

In [288]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(x_sm, y_sm)

#모델 검증
print('training data fit :', log_model.score(x_sm, y_sm)) 
print('test data fit :', log_model.score(x_test, y_test)) 

training data fit : 0.7313642756680732
test data fit : 0.6034271725826194


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [290]:
# 그리드 서치
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), categorical_features), remainder='passthrough')
# # transformer = make_pipeline(transformer, MaxAbsScaler())

# pipe = Pipeline([('scaler', transformer),
#                  ('smote', SMOTE()),
#                 ('model', SVC())])

param_grid = [{'penalty' : ['l1', 'l2'],
               'C' : [2, 2.2, 2.4, 2.6, 2.8]}]

gs = GridSearchCV(estimator = LogisticRegression(), param_grid = param_grid,
                  scoring = 'accuracy', cv = 10, n_jobs = -1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'C': 2.8, 'penalty': 'l2'}
최적 점수: 0.7111218186310848
LogisticRegression(C=2.8)


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueEr

In [291]:
#모델 학습
log_model = gs.best_estimator_

In [292]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1) # out 데이터에서 '당뇨병' 칼럼만 제거
x_test

,생년,월 혈압약 일수,장애주된 원인,질병명,키,몸무게,현재 지속적 진료여부,주관적 건강 평가,물리치료-이용여부,평균음주량,...,이용_만성질환관리,ADL_보행,주로 도와주는 사람,상지의지-소지,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
27,1944,0,69,0,168.0,74.0,1,4,2,1,...,1,1,3,2,2,2,0,0,0,2


In [293]:
labels = ['무', '유']

In [295]:
x_test = transformer.transform(x_test)

y_predict = log_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = log_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.7979238895716176
